In [1]:
!pip install seaborn

!pip install git+https://github.com/tensorflow/docs

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-1u2k3lt3
  Running command git clone -q https://github.com/tensorflow/docs /tmp/pip-req-build-1u2k3lt3
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.0-cp36-none-any.whl size=80383 sha256=836d339619c4b2f596802c76c074a4967c0aa48330243ccaa835370be0b84f20
  Stored in directory: /tmp/pip-ephem-wheel-cache-p80pzrqo/wheels/eb/1b/35/fce87697be00d2fc63e0b4b395b0d9c7e391a10e98d9a0d97f
Successfully built tensorflow-docs


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [4]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

TensorFlow 2.x selected.
2.1.0


In [0]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from sklearn.preprocessing import LabelEncoder

In [6]:
dataset = pd.read_csv('Train_v2.csv')
dataset.head(10)

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,No,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,No,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed
5,Kenya,2018,uniqueid_6,No,Rural,No,7,26,Female,Spouse,Married/Living together,Primary education,Informally employed
6,Kenya,2018,uniqueid_7,No,Rural,Yes,7,32,Female,Spouse,Married/Living together,Primary education,Self employed
7,Kenya,2018,uniqueid_8,No,Rural,Yes,1,42,Female,Head of Household,Married/Living together,Tertiary education,Formally employed Government
8,Kenya,2018,uniqueid_9,Yes,Rural,Yes,3,54,Male,Head of Household,Married/Living together,Secondary education,Farming and Fishing
9,Kenya,2018,uniqueid_10,No,Urban,Yes,3,76,Female,Head of Household,Divorced/Seperated,No formal education,Remittance Dependent


In [5]:
dataset.isna().sum()

country                   0
year                      0
uniqueid                  0
bank_account              0
location_type             0
cellphone_access          0
household_size            0
age_of_respondent         0
gender_of_respondent      0
relationship_with_head    0
marital_status            0
education_level           0
job_type                  0
dtype: int64

In [0]:
dataset.pop("uniqueid")
dataset.pop("year")
dataset['age_of_respondent'] = pd.to_numeric(dataset['age_of_respondent'], errors='coerce')
dataset['household_size'] = pd.to_numeric(dataset['household_size'], errors='coerce')

In [8]:
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='', columns=["country", "relationship_with_head", "marital_status", "education_level", "job_type"])
labelencoder = LabelEncoder()
dataset.iloc[:, 0] = labelencoder.fit_transform(dataset.iloc[:, 0].values)
dataset.iloc[:, 1] = labelencoder.fit_transform(dataset.iloc[:, 1].values)
dataset.iloc[:, 2] = labelencoder.fit_transform(dataset.iloc[:, 2].values)
dataset.iloc[:, 5] = labelencoder.fit_transform(dataset.iloc[:, 5].values)
dataset.tail()

,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,Kenya,Rwanda,Tanzania,Uganda,Child,Head of Household,Other non-relatives,Other relative,Parent,Spouse,Divorced/Seperated,Dont know,Married/Living together,Single/Never Married,Widowed,No formal education,Other/Dont know/RTA,Primary education,Secondary education,Tertiary education,Vocational/Specialised training,Dont Know/Refuse to answer,Farming and Fishing,Formally employed Government,Formally employed Private,Government Dependent,Informally employed,No Income,Other Income,Remittance Dependent,Self employed
23519,0,0,1,4,48,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
23520,0,0,1,2,27,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
23521,0,0,1,5,27,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
23522,0,1,1,7,30,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
23523,0,0,1,10,20,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0


In [0]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23524 entries, 0 to 23523
Data columns (total 40 columns):
bank_account                       23524 non-null int64
location_type                      23524 non-null int64
cellphone_access                   23524 non-null int64
household_size                     23524 non-null int64
age_of_respondent                  23524 non-null int64
gender_of_respondent               23524 non-null int64
Kenya                              23524 non-null uint8
Rwanda                             23524 non-null uint8
Tanzania                           23524 non-null uint8
Uganda                             23524 non-null uint8
2016                               23524 non-null uint8
2017                               23524 non-null uint8
2018                               23524 non-null uint8
Child                              23524 non-null uint8
Head of Household                  23524 non-null uint8
Other non-relatives                23524 non-null u

In [0]:
dataset.describe()

,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,Kenya,Rwanda,Tanzania,Uganda,2016,2017,2018,Child,Head of Household,Other non-relatives,Other relative,Parent,Spouse,Divorced/Seperated,Dont know,Married/Living together,Single/Never Married,Widowed,No formal education,Other/Dont know/RTA,Primary education,Secondary education,Tertiary education,Vocational/Specialised training,Dont Know/Refuse to answer,Farming and Fishing,Formally employed Government,Formally employed Private,Government Dependent,Informally employed,No Income,Other Income,Remittance Dependent,Self employed
count,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000,23524.000000
mean,0.140792,0.390282,0.741966,3.797483,38.805220,0.410092,0.257949,0.371323,0.281415,0.089313,0.371323,0.281415,0.347262,0.094754,0.545443,0.008077,0.028397,0.046166,0.277164,0.088250,0.000340,0.456938,0.339356,0.115116,0.191932,0.001488,0.543743,0.179519,0.049184,0.034135,0.005356,0.231296,0.016451,0.044848,0.010500,0.237927,0.026654,0.045911,0.107422,0.273635
std,0.347815,0.487824,0.437562,2.227613,16.520569,0.491861,0.437515,0.483169,0.449699,0.285201,0.483169,0.449699,0.476110,0.292881,0.497941,0.089510,0.166106,0.209848,0.447608,0.283665,0.018438,0.498153,0.473501,0.319169,0.393828,0.038545,0.498094,0.383794,0.216256,0.181581,0.072992,0.421670,0.127206,0.206974,0.101932,0.425823,0.161072,0.209296,0.309656,0.445834
min,0.000000,0.000000,0.000000,1.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,2.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,3.000000,35.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,1.000000,5.000000,49.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,21.000000,100.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
output = dataset.pop("bank_account")
X_train = dataset
Y_train = output

In [0]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)

In [0]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[36]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [0]:
model = build_model()

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                2368      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 6,593
Trainable params: 6,593
Non-trainable params: 0
_________________________________________________________________


In [14]:

example_batch = X_train[:10]

example_result = model.predict(example_batch)
example_result

array([[-0.07873997],
       [ 0.5175238 ],
       [-0.570799  ],
       [ 0.5792637 ],
       [-0.06980962],
       [-0.22194955],
       [-0.05885685],
       [-0.19585076],
       [-0.08380014],
       [ 0.1746152 ]], dtype=float32)

In [0]:
EPOCHS = 500

history = model.fit(
  X_train, Y_train,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[tfdocs.modeling.EpochDots()])


Epoch: 0, accuracy:0.8431,  loss:0.9701,  val_accuracy:0.9103,  val_loss:0.7099,  
....................................................................................................
Epoch: 100, accuracy:0.8894,  loss:0.3261,  val_accuracy:0.9239,  val_loss:0.6444,  
....................................................................................................
Epoch: 200, accuracy:0.9022,  loss:0.2877,  val_accuracy:0.9222,  val_loss:0.8017,  
....................................................................................................
Epoch: 300, accuracy:0.9027,  loss:0.3077,  val_accuracy:0.9197,  val_loss:0.8007,  
....................................................................................................
Epoch: 400, accuracy:0.9165,  loss:0.2606,  val_accuracy:0.9150,  val_loss:0.9128,  
...........................................................................

In [0]:
dataset_test = pd.read_csv('Test_v2.csv')
dataset_test.head(10)

,country,year,uniqueid,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_6056,Urban,Yes,3,30,Male,Head of Household,Married/Living together,Secondary education,Formally employed Government
1,Kenya,2018,uniqueid_6060,Urban,Yes,7,51,Male,Head of Household,Married/Living together,Vocational/Specialised training,Formally employed Private
2,Kenya,2018,uniqueid_6065,Rural,No,3,77,Female,Parent,Married/Living together,No formal education,Remittance Dependent
3,Kenya,2018,uniqueid_6072,Rural,No,6,39,Female,Head of Household,Married/Living together,Primary education,Remittance Dependent
4,Kenya,2018,uniqueid_6073,Urban,No,3,16,Male,Child,Single/Never Married,Secondary education,Remittance Dependent
5,Kenya,2018,uniqueid_6074,Rural,Yes,4,25,Female,Spouse,Married/Living together,Primary education,Self employed
6,Kenya,2018,uniqueid_6075,Rural,Yes,4,32,Female,Head of Household,Divorced/Seperated,Primary education,Informally employed
7,Kenya,2018,uniqueid_6076,Urban,Yes,3,24,Female,Spouse,Married/Living together,Vocational/Specialised training,Self employed
8,Kenya,2018,uniqueid_6077,Rural,Yes,4,22,Female,Child,Single/Never Married,Primary education,Informally employed
9,Kenya,2018,uniqueid_6078,Urban,Yes,1,42,Male,Head of Household,Divorced/Seperated,Secondary education,Farming and Fishing


In [0]:
array_id = dataset_test.pop("uniqueid")
array_country = dataset_test['country']
dataset_test.pop("year")

In [0]:
dataset_test['age_of_respondent'] = pd.to_numeric(dataset['age_of_respondent'], errors='coerce')
dataset_test['household_size'] = pd.to_numeric(dataset['household_size'], errors='coerce')

In [0]:
dataset_test = pd.get_dummies(dataset_test, prefix='', prefix_sep='', columns=["country", "relationship_with_head", "marital_status", "education_level", "job_type"])
labelencoder = LabelEncoder()
dataset_test.iloc[:, 0] = labelencoder.fit_transform(dataset_test.iloc[:, 0].values)
dataset_test.iloc[:, 1] = labelencoder.fit_transform(dataset_test.iloc[:, 1].values)
dataset_test.iloc[:, 4] = labelencoder.fit_transform(dataset_test.iloc[:, 4].values)
dataset_test.tail()

,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,Kenya,Rwanda,Tanzania,Uganda,2016,2017,2018,Child,Head of Household,Other non-relatives,Other relative,Parent,Spouse,Divorced/Seperated,Dont know,Married/Living together,Single/Never Married,Widowed,No formal education,Other/Dont know/RTA,Primary education,Secondary education,Tertiary education,Vocational/Specialised training,Dont Know/Refuse to answer,Farming and Fishing,Formally employed Government,Formally employed Private,Government Dependent,Informally employed,No Income,Other Income,Remittance Dependent,Self employed
10081,0,0,6,64,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
10082,1,1,4,30,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
10083,1,1,4,27,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
10084,0,1,6,40,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
10085,1,1,3,26,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0


In [0]:
X_test = sc_X.fit_transform(dataset_test)

In [0]:
Y_pred = model.predict(X_test).flatten()

In [0]:
file = open("NN.txt","w") 
file.write("uniqueid,bank_account\n")

for i in range(0, len(X_test)):
    if (Y_pred[i]<0.5):
      result = 0
    else:
      result = 1
    

    file.write(array_id[i]+" x "+array_country[i]+","+str(result)+"\n")
    
file.close()